## Importing the packages

In [ ]:
import numpy as np

In [ ]:
#Let's make the displayed data more viewer-friendly using the code below
np.set_printoptions(suppress = True, linewidth = 100, precision = 2)

Importing the Data

In [ ]:
# To ascertain if there are any values let's use np.loadtxt, an error means there are missing values
raw_data_np = np.loadtxt("loan-data.csv", delimiter = ';')
raw_data_np

In [ ]:
# let's switch to np.genfromtxt since we got errors above there are missing values
raw_data_np = np.genfromtxt("loan-data.csv", delimiter = ';')
raw_data_np

In [ ]:
# let's use skip_header abnd autostrip because the entire first row has nan values and whitespaces that distort our columns respectively
raw_data_np = np.genfromtxt("loan-data.csv", delimiter = ';',
                            skip_header = 1,
                            autostrip = True)
raw_data_np

## Checking for incomplete data

In [ ]:
# Let dot chain the sum() with isnan() to determine the number of missing values
np.isnan(raw_data_np).sum()

In [ ]:
# We have lots of missing values so let's take care of them. temporary_fill will be greater
#than any values in the data set and thus serves as a filler for all missing entries.temporary_mean is the mean for each column

temporary_fill = np.nanmax(raw_data_np).round(2) +1
temporary_mean = np.nanmean(raw_data_np, axis = 0)

In [ ]:
# We got a warning  (RuntimeWarning: Mean of empty slice) after running the previsous code 
#because some columns consist entriely of text data hence the means are NAN.We will have to 
#split the data into numerical and nonnumerical data

temporary_mean

In [ ]:
temporary_stats = np.array([np.nanmin(raw_data_np, axis = 0),
                           temporary_mean,
                           np.nanmax(raw_data_np, axis = 0)])

In [ ]:
temporary_stats

## Splitting the Dataset

In [ ]:
#We first need to know which columns contain numerical data (columns_numeric) and nonnumerical data (column_strings),we can use temporary_mean
# and use np.argwhere to return the indices of 'nan' columns
columns_strings = np.argwhere(np.isnan(temporary_mean) == True).squeeze()
columns_strings

In [ ]:
# Let's identify columns with numeric data
columns_numeric = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
columns_numeric

## Re-importing the dataset

In [ ]:
#Let's re-import the data using the usecols parameter to split the data into two arrays (numeric and nonnumeric data)
# rename variables appropriately

loan_data_strings = np.genfromtxt("loan-data.csv", delimiter = ';',
                            skip_header = 1,
                            autostrip = True,
                            usecols = columns_strings,
                            dtype = np.str_)
loan_data_strings 

In [ ]:
#Follow the methodology above and copy the code and rename it to make the numerical array

loan_data_numeric = np.genfromtxt("loan-data.csv", delimiter = ';',
                            skip_header = 1,
                            autostrip = True,
                            usecols = columns_numeric,
                            filling_values = temporary_fill)
loan_data_numeric

## The Names of the Columns

In [ ]:
#Let's get all the names of the columns
header_full = np.genfromtxt("loan-data.csv",
                            delimiter = ';',
                            skip_footer = raw_data_np.shape[0],
                            autostrip = True,
                            dtype = np.str_)
header_full 

In [ ]:
header_strings, header_numeric = header_full[columns_strings],header_full[columns_numeric]

In [ ]:
header_strings

In [ ]:
header_numeric

## Creating Checkpoints:

In [ ]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)
    

In [ ]:
checkpoint_test = checkpoint("checkpoint-test", header_strings, loan_data_strings)

In [ ]:
checkpoint_test['header']

In [ ]:
checkpoint_test['data']

In [ ]:
np.array_equal(checkpoint_test['data'], loan_data_strings)

## Manipulating String Columns

In [ ]:
#Let's recall the header names of columns containing only string values
header_strings

In [ ]:
#let's rename some columns to make it easier to know what they represent

header_strings[0] = "issue_date"

In [ ]:
# Let's look at the string data in its entirety

loan_data_strings

## Issue Date

In [ ]:
# We can manipulate the issue date data so let's look at that column
loan_data_strings[:,0]

In [ ]:
# Months are represented by the first three letters of the month.Let's confirm if this is pattern for the whole column
np.unique(loan_data_strings[:,0])

In [ ]:
#Looks like all the data contains '-15' in common so we can strip it away without losing insghtful information
np.chararray.strip(loan_data_strings[:,0], "-15")

In [ ]:
#Let's overwirte the data in that column since we confirmed in above cell that the '-15' has been removed for each element
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], "-15")


In [ ]:
#Let's look at the unique elements again to confrim we have removed excess data ('-15')
np.unique(loan_data_strings[:,0])

In [ ]:
#In analysis it is more convenient to represent the month values as integers
# One advatage is that this Store the data using less memory or space
#Enables a more easy to follow order of the months
# Let's use a 'for loop' and months 'array' to change the month strings to string 'values' first and then cast into integers later

months  = np.array(['','Jan', 'Feb','Mar','Apr', 'May','Jun','Jul', 'Aug','Sep','Oct', 'Nov','Dec'])

In [ ]:
for i in range(13):
    loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                      i,
                                     loan_data_strings[:,0])


In [ ]:
#Let's confirm that there are no more str for month data
np.unique(loan_data_strings[:,0])

## Loan Status



In [ ]:
#Let's examine other columns to find any excess data
header_strings

In [ ]:
loan_data_strings[:,1]

In [ ]:
#Let's confirm if all elements in this column are 'current'

np.unique(loan_data_strings[:,1])

In [ ]:
np.unique(loan_data_strings[:,1]).size

In [ ]:
#Banks use regression analysis to assess loan applicants, their primary concern is 
#determining whether the applicant has a consistent and dependable financial standing. 
#We need to split all possible values of 'loan_status' into either good or bad (0 or 1)
#fully paid, Current, In Grace period, Issued, late (16-30 days) = good and the rest are bad

status_bad = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])

In [ ]:
#Let's check if an element in loan_status column is in status_bad, if so assign it a zero and if not, assign a one
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], status_bad), 0, 1)

In [ ]:
#let's confirm if all elements in the loan_status have the dumby variable 1 or 0 only
np.unique(loan_data_strings[:,1])

## Term

In [ ]:
#Let's ;ool at other columns in the string data
header_strings

In [ ]:
loan_data_strings[:,2]

In [ ]:
np.unique(loan_data_strings[:,2])

In [ ]:
#let's remove the " months" (empty months) string

loan_data_strings[:,2] = np.chararray.strip(loan_data_strings[:,2], " months")
loan_data_strings[:,2]

In [ ]:
np.unique(loan_data_strings[:,2])

In [ ]:
#Let's rename the column to make it more descriptive of what the data represents
header_strings[2] = "term_months"

In [ ]:
#When we have missing data in the Credit Risk Model (CRM), we assume the worst
#Let's convert empty strings to '60'
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '',
                                  '60', 
                                  loan_data_strings[:,2])


In [ ]:
#Let's check to see the contents of term_months again
#After that we will look at grade and subgrade
np.unique(loan_data_strings[:,2])

Grade and Subgrade

In [ ]:
header_strings

In [ ]:
loan_data_strings[:,3]

In [ ]:
np.unique(loan_data_strings[:,3])

In [ ]:
loan_data_strings[:,4]


In [ ]:
np.unique(loan_data_strings[:,4])

In [ ]:
#Grade and subgrade are  related making grade somewhat redundant, thus can grade can be later removed
# Empty strings in subgrade can be substituted using grade
# Use grade for an approriate approximation of subgrade empty string

for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                      i + '5',
                                      loan_data_strings[:,4])

In [ ]:
np.unique(loan_data_strings[:,4])

In [ ]:
#When return_counts = True, np.unique will return the number of times each unique value occurs in the input array.
#Let's see how many times we get the empty string
np.unique(loan_data_strings[:,4], return_counts = True)

In [ ]:
#We have 9 empty spaces out of 10 000 rows which means they are insignificant
#However, let's not igonore these empty space, instead
#Let's assign the empty space a new value of 'H1'

loan_data_strings[:,4] = np.where(loan_data_strings[:,4] == '',
                                      'H1',
                                      loan_data_strings[:,4])

In [ ]:
np.unique(loan_data_strings[:,4])

Removing Grade

In [ ]:
#We no longer need grade in the dataset as the information is carried by subgrade
#axis = 1 indicates that the deletion operation should be applied along the columns (or second axis) of the array.
loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)

In [ ]:
loan_data_strings[:,3]

In [ ]:
header_strings

In [ ]:
header_strings =np.delete(header_strings, 3)

In [ ]:
header_strings

In [ ]:
header_strings[3]

## Converting Sub Grade

In [ ]:
loan_data_strings[:,3]

In [ ]:
np.unique(loan_data_strings[:,3])

In [ ]:
#Let's convert the sub-grade elements into numeric values

keys = list(np.unique(loan_data_strings[:,3]))
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1))
dict_sub_grade  = dict(zip(keys, values))

In [ ]:
dict(zip(keys, values))

In [ ]:
for i in np.unique(loan_data_strings[:,3]):
    loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i,
                                      dict_sub_grade[i],
                                     loan_data_strings[:,3])

In [ ]:
np.unique(loan_data_strings[:,3])

## Verification Status

In [ ]:
header_strings

In [ ]:
loan_data_strings[:,4]

In [ ]:
np.unique(loan_data_strings[:,4])

In [ ]:
#Just line lon status, we will set the good and bad verification_statuses as 1 and 0 repectively
# '' and not verified are bad hence equal to 0
# source verified and verified are good hence assign a 1


loan_data_strings[:, 4] = np.where((loan_data_strings[:, 4] == '') | (loan_data_strings[:, 4] == 'Not Verified'), 0, 1)

In [ ]:
np.unique(loan_data_strings[:,4])

## URL

In [ ]:
header_strings

In [ ]:
loan_data_strings[:,5]

In [ ]:
np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [ ]:
loan_data_strings[:,5] = np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [ ]:
np.unique(loan_data_strings[:,5])

In [ ]:
# Recall that the id in the numeric string might possibly match this last digits on each url

header_full

In [ ]:
#call the first column from the numeric dataset
loan_data_numeric[:,0]

In [ ]:
#call the URL column

loan_data_strings[:,5]

In [ ]:
#cast the URL data into integers
loan_data_strings[:,5].astype(dtype = np.int32)

In [ ]:
#verify that the id and url suffix are the same using np. array_equal
#Results show that the URL column doesnt hold any information which we cannot extract from the ID column so we can delete it
np.array_equal(loan_data_strings[:,5].astype(dtype = np.int32),loan_data_numeric[:,0].astype(dtype = np.int32))

In [ ]:
#Let's get rid of url column from the header and string data
loan_data_strings = np.delete(loan_data_strings, 5, axis  = 1)
header_strings = np.delete(header_strings, 5)

In [ ]:
#In the next cells just comfirm that we didn't accidentally delete important data (ID) from the numeric dataset
loan_data_strings[:,5]

In [ ]:
header_strings

In [ ]:
loan_data_numeric[:,0]

In [ ]:
header_numeric

## State Address

In [ ]:
header_strings

In [ ]:
loan_data_strings[:,5]

In [ ]:
np.unique(loan_data_strings[:,5])

In [ ]:
np.unique(loan_data_strings[:,5]).size

In [ ]:
header_strings[5] = "state_address"

In [ ]:
header_strings

In [ ]:
np.unique(loan_data_strings[:,5], return_counts = True)

In [ ]:
#Let's sort states and matching number of applicants in descing order
#argsort will give us the indices of values/counts arranged in ascending order so use a negative sign for descending order
states_names, states_count = np.unique(loan_data_strings[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)
states_names[states_count_sorted], states_count[states_count_sorted]


In [ ]:
#There are more applications with missing or unreported addresses than there are for 45 of the other states
# i.e 500 APPLICANTS from empty string state ''
#Aggregated geographical data (e.g., regional economic indicators) 
#may help address the limitations associated with missing location information while still capturing relevant geographical risk factors.
# First let's take care of empty valuessss

loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == '', 0, loan_data_strings[:,5] )

## https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [ ]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [ ]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [ ]:
np.unique(loan_data_strings[:,5])

In [ ]:
header_strings


Converting to Numbers

In [ ]:
loan_data_strings

In [ ]:
loan_data_strings = loan_data_strings.astype(int)

In [ ]:
loan_data_strings

## Checkpoint 1: Strings

In [ ]:
checkpoint_strings = checkpoint("checkpoint-strings", header_strings, loan_data_strings)

In [ ]:
checkpoint_strings["header"]

In [ ]:
checkpoint_strings["data"]

In [ ]:
np.array_equal(checkpoint_strings["data"],loan_data_strings)

Numerical Data

In [ ]:
loan_data_numeric

In [ ]:
#Technically speaing there aren't any missing values in the array because of temporary_fill
np.isnan(loan_data_numeric).sum()

## Substitute 'filler' values

In [ ]:
header_numeric

ID

In [ ]:
temporary_fill

In [ ]:
#Let's check if we have filled any missing values in the ID column
np.isin(loan_data_numeric[:,0], temporary_fill)

In [ ]:
np.isin(loan_data_numeric[:,0], temporary_fill).sum()

In [ ]:
header_numeric


In [ ]:
#Funded_amnt missing entries have to be substituted with the lowest recorded values
# loan_amnt, int_rate,installment, total_pymnt missing entries need to be replaced with maximum recorded values



## Temporary Stats

In [ ]:
temporary_stats

In [ ]:
temporary_stats[:,columns_numeric]

## Funded Amount

In [ ]:
loan_data_numeric[:,2]

In [ ]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill,
                                 temporary_stats[0, columns_numeric[2]],
                                 loan_data_numeric[:,2])
loan_data_numeric[:,2]

In [ ]:
temporary_stats[0, columns_numeric[3]]

## Loaned Amount, Interest Rate, Total Payment, Installment

In [ ]:
header_numeric

In [ ]:
for i in [1, 3, 4, 5]:
    loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == temporary_fill,
                                 temporary_stats[2, columns_numeric[i]],
                                 loan_data_numeric[:,i])

In [ ]:
loan_data_numeric

Currency Exchange

The exchange rate

In [ ]:
EUR_USD = np.genfromtxt("EUR-USD.csv", delimiter = ',', autostrip = True)
EUR_USD

In [ ]:
EUR_USD = np.genfromtxt("EUR-USD.csv", delimiter = ',', autostrip = True, dtype = np.str_)
EUR_USD

In [ ]:
EUR_USD = np.genfromtxt("EUR-USD.csv", delimiter = ',', autostrip = True, skip_header = 1, usecols = 3)
EUR_USD

In [ ]:
#Excahnge rate value must match the month the loan was issued.
# Issue date is the first column from loan_data_strings dataset
loan_data_strings[:,0]

In [ ]:
exchange_rate = loan_data_strings[:,0]

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    

exchange_rate = np.where(exchange_rate == 0,
                         np.mean(EUR_USD),
                         exchange_rate)

exchange_rate

In [ ]:
#Exchange rate column must be added to the numeric data
#check for compatability of the two arrays
exchange_rate.shape

In [ ]:
loan_data_numeric.shape

In [ ]:
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [ ]:
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))

In [ ]:
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))
header_numeric

In [ ]:
header_numeric

## FROM USD to EUR

In [ ]:
header_numeric

In [ ]:
columns_dollar = np.array([1,2,4,5])

In [ ]:
loan_data_numeric[:,[columns_dollars]]

In [ ]:
loan_data_numeric[:,6]

In [ ]:
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

In [ ]:
loan_data_numeric

In [ ]:
loan_data_numeric.shape

## Expanding the header

In [ ]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [ ]:
header_additional

In [ ]:
header_numeric = np.concatenate((header_numeric, header_additional))

In [ ]:
header_numeric

In [ ]:
header_numeric[columns_dollar] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollar]])

In [ ]:
header_numeric

In [ ]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [ ]:
header_numeric = header_numeric[columns_index_order]

In [ ]:
loan_data_numeric

## Interest Rate

In [ ]:
header_numeric

In [ ]:
loan_data_numeric[:,5]

In [ ]:
loan_data_numeric[:,5] = loan_data_numeric[:,5]/100

In [ ]:
loan_data_numeric[:,5]

## Checkpoint 2: Numeric

In [ ]:
checkpoint_numeric = checkpoint("Checkpoint-Numeric", header_numeric, loan_data_numeric)

In [ ]:
checkpoint_numeric['header'], checkpoint_numeric['data']

## Creating the "Complete" Dataset

In [ ]:
#Let's check our datasets for compatibility in terms of stacking
loan_data_numeric.shape

In [ ]:
loan_data_strings.shape

## Safer option to check is to use checkpoint files

In [ ]:
checkpoint_strings['data'].shape

In [ ]:
checkpoint_numeric['data'].shape

In [ ]:
#Datasets contain the same number of rows so they can be horizontally stacked
np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))

In [ ]:
np.hstack((checkpoint_numeric['data'], checkpoint_strings['data'])).shape

In [ ]:
loan_data = np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))

In [ ]:
loan_data


In [ ]:
#Let's check for missing values 
#We expect to see O since we took care of them earlier in the practical example
np.isnan(loan_data).sum()

In [ ]:
np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

In [ ]:
header_full_new = np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

In [ ]:
header_full_new

## Sorting the new dataset

In [ ]:
#Last step let's sort using the first 'id' column from loan_data
np.sort(loan_data[:,0])

In [ ]:
#Let's use argsort to get the indices
loan_data[np.argsort(loan_data[:,0])]

In [ ]:
loan_data = loan_data[np.argsort(loan_data[:,0])]

In [ ]:
loan_data

In [ ]:
np.argsort(loan_data[:,0])

## Storing the New Dataset

In [ ]:
#Let's complete the dataset by stacking the header for the loan data using vstack
loan_data = np.vstack((header_full_new, loan_data))

In [ ]:
loan_data

In [ ]:
np.savetxt("loan-data-preprocessed.csv", 
           loan_data, 
           fmt = '%s',
           delimiter = ',')